python -m venv myenv

.\myenv\Scripts\activate

pip install -r requirements.txt


pip install openpyxl


In [5]:
import pandas as pd
import numpy as np

import faiss

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from tqdm import tqdm
import time

import ipywidgets as widgets
widgets.IntSlider()

query= "what is the max rating given in home and lifestyle?"


In [3]:
df = pd.read_csv("output_table.csv")

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
column_names = df.columns.to_list()
column_name_embeddings = model.encode(column_names)

In [6]:
column_index_mapping = {
    "invoice id": 0,
    "city": 1,
    "gender": 2,
    "product line": 3,
    "unit price": 4,
    "quantity": 5,
    "total": 6,
    "date": 7,
    "payment": 8,
    "gross income": 9,
    "rating": 10
}


In [7]:
column_name_mapping = {
    'city' : df["city"].unique().tolist(),
    'gender' : df["gender"].unique().tolist(),
    'product line': df["product line"].unique().tolist(),
    'payment' : df["payment"].unique().tolist()
    
}

In [8]:
column_name_mapping

{'city': ['Yangon', 'Naypyitaw', 'Mandalay'],
 'gender': ['Female', 'Male'],
 'product line': ['Health beauty',
  'Electronic accessories',
  'Home lifestyle',
  'Sports travel',
  'Food beverages',
  'Fashion accessories'],
 'payment': ['Ewallet', 'Cash', 'Credit card']}

In [9]:
column_name_embeddings

array([[-0.1117169 ,  0.11272218, -0.02423377, ...,  0.00350045,
         0.03952534, -0.05215904],
       [ 0.05124492,  0.07561278, -0.03347927, ...,  0.02809969,
        -0.03862095,  0.05421344],
       [ 0.02549297,  0.05705191, -0.0425216 , ...,  0.01834778,
         0.07324108, -0.05377625],
       ...,
       [-0.04512803,  0.09326787, -0.01289233, ..., -0.03927517,
         0.05541484, -0.10761316],
       [ 0.02171222,  0.0022199 ,  0.00923696, ..., -0.05872287,
         0.03283434, -0.12940843],
       [-0.06318381, -0.00255952, -0.12002307, ..., -0.06058924,
        -0.00833742,  0.05499737]], dtype=float32)

In [11]:
def preprocess_query(user_query):
    # Filter out stop words from the user query
    filtered_text = [word for word in user_query.split() if word.lower() not in ENGLISH_STOP_WORDS]
    
    # Lemmatize each word in the filtered text to get its base form
    lemmetized_text = [WordNetLemmatizer().lemmatize(word) for word in filtered_text]
    
    # Return the preprocessed list of words
    return lemmetized_text


In [12]:
#column filtering
def get_matching_columns(list_query, threshold=0.7):
    matched_columns = []

    for query in list_query:
        query_embedding = model.encode([query])
        
        # Check similarity with column names
        similarities = cosine_similarity(query_embedding, column_name_embeddings)
        top_column_indices = np.argsort(similarities[0])[::-1]
        
        added = False
        for idx in top_column_indices:
            if similarities[0][idx] >= threshold:
                matched_columns.append(column_names[idx])
                added = True
                break
        
        # Fallback: Check similarity with column values using embeddings
        if not added:
            for column, values in column_name_mapping.items():
                value_embeddings = model.encode(values)
                value_similarities = cosine_similarity(query_embedding, value_embeddings)
                
                # Check if any value matches the threshold
                max_similarity = value_similarities.max()
                if max_similarity >= threshold:
                    matched_columns.append(column)
                    break

    # Deduplicate the column list
    return list(set(matched_columns))

In [13]:
final_query = preprocess_query(query) # Example query

In [14]:
final_query  # Example query after preprocessing

['max', 'rating', 'given', 'home', 'lifestyle?']

In [15]:
get_matching_columns(final_query) # Queries that match the columns 

['rating', 'product line']

In [ ]:
QA = pd.read_excel("QA_dataset_share.xlsx")  # Excel Q/A data provided for testing

In [18]:
QA.head()

,question,row index,column index,answer,filtered row index,filtered column index,generated response
0,What product line is in the latest entry?,999,3,Fashion accessories,NaN,NaN,NaN
1,On what date did the first transaction occur?,"17, 245, 450, 484, 496, 523, 567, 696, 829, 83...",7,1/1/2019,NaN,NaN,NaN
2,What is the latest transaction date?,"158, 306, 473, 474, 643, 646, 671, 881, 883, 9...",7,3/30/2019,NaN,NaN,NaN
3,what is the max rating given in home and lifes...,"2, 7, 19, 22, 25, 39, 40, 41, 54, 56, 58, 61, ...","3,10",9.9,NaN,NaN,NaN
4,How many transactions involved Male customers ...,"331, 464, 540, 708, 710","2,10",5,NaN,NaN,NaN


In [19]:
QA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   question               70 non-null     object 
 1   row index              70 non-null     object 
 2   column index           70 non-null     object 
 3   answer                 70 non-null     object 
 4   filtered row index     0 non-null      float64
 5   filtered column index  0 non-null      float64
 6   generated response     0 non-null      float64
dtypes: float64(3), object(4)
memory usage: 4.0+ KB


In [21]:
def update_filtered_column_indices(df, column_name_mapping):
    """
    Updates the DataFrame by adding a 'filtered column index' column
    based on mapped question-to-column indices.

    Parameters:
    df (pd.DataFrame): Input DataFrame with a 'question' column.
    column_name_mapping (dict): Dictionary mapping column names to their indices.

    Returns:
    pd.DataFrame: DataFrame with an updated 'filtered column index' column.
    """
    
    def map_question_to_indices(question):
        """
        Maps a given question to column indices based on matching columns.

        Parameters:
        question (str): A question string to be processed.

        Returns:
        list: List of column indices corresponding to matching columns.
        """
        # Preprocess the question to normalize or clean it
        final_query = preprocess_query(question)
        
        # Retrieve column names that match the processed query
        matched_columns = get_matching_columns(final_query)
        
        # Map the matched columns to their corresponding indices
        return [column_name_mapping[col] for col in matched_columns if col in column_name_mapping]
    
    # Enable the progress bar for DataFrame operations
    tqdm.pandas(desc="Processing questions")
    
    # Apply the mapping function to each question in the DataFrame and store the result in a new column
    df["filtered column index"] = df["question"].progress_apply(lambda q: map_question_to_indices(q))
    
    # Return the updated DataFrame
    return df


In [22]:
QA = update_filtered_column_indices(QA, column_index_mapping)
print(QA)


Processing questions: 100%|██████████| 70/70 [00:13<00:00,  5.00it/s]

                                             question  \
0           What product line is in the latest entry?   
1       On what date did the first transaction occur?   
2                What is the latest transaction date?   
3   what is the max rating given in home and lifes...   
4   How many transactions involved Male customers ...   
..                                                ...   
65                          What is the minimum cost?   
66  How many transactions involved Ewallet payment...   
67  What is the total gross income for transaction...   
68  How many transactions involved Female customer...   
69  What is the total gross income for transaction...   

                                            row index column index  \
0                                                 999            3   
1   17, 245, 450, 484, 496, 523, 567, 696, 829, 83...            7   
2   158, 306, 473, 474, 643, 646, 671, 881, 883, 9...            7   
3   2, 7, 19, 22, 25, 39, 40, 41, 5

In [23]:
QA

,question,row index,column index,answer,filtered row index,filtered column index,generated response
0,What product line is in the latest entry?,999,3,Fashion accessories,NaN,[3],NaN
1,On what date did the first transaction occur?,"17, 245, 450, 484, 496, 523, 567, 696, 829, 83...",7,1/1/2019,NaN,[7],NaN
2,What is the latest transaction date?,"158, 306, 473, 474, 643, 646, 671, 881, 883, 9...",7,3/30/2019,NaN,[7],NaN
3,what is the max rating given in home and lifes...,"2, 7, 19, 22, 25, 39, 40, 41, 54, 56, 58, 61, ...","3,10",9.9,NaN,"[10, 3]",NaN
4,How many transactions involved Male customers ...,"331, 464, 540, 708, 710","2,10",5,NaN,"[10, 2]",NaN
...,...,...,...,...,...,...,...
65,What is the minimum cost?,"0,\n 1,\n 2,\n 3,\n 4,\n 5,\n 6,\n 7,\n...",6,10.67,NaN,[],NaN
66,How many transactions involved Ewallet payment...,"12,\n 20,\n 23,\n 36,\n 116,\n 157,\n 17...","5,8",36,NaN,"[8, 5]",NaN
67,What is the total gross income for transaction...,"7, 362, 582, 656, 699, 771","1,9,10",139.84,NaN,"[10, 9, 1, 6]",NaN
68,How many transactions involved Female customer...,"1, 12, 28, 51, 79, 126, 136, 199, 216, 237, 23...","2, 5",56,NaN,"[5, 2]",NaN


In [24]:
QA.to_excel("updated_QA.xlsx", index=False)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   invoice id    1000 non-null   object 
 1   city          1000 non-null   object 
 2   gender        1000 non-null   object 
 3   product line  1000 non-null   object 
 4   unit price    1000 non-null   float64
 5   quantity      1000 non-null   int64  
 6   total         1000 non-null   float64
 7   date          1000 non-null   object 
 8   payment       1000 non-null   object 
 9   gross income  1000 non-null   float64
 10  rating        1000 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 86.1+ KB


### Row filtering

In [2]:
from typing import List

In [6]:
updated_qa = pd.read_excel("updated_QA.xlsx")
updated_qa.head()

,question,row index,column index,answer,filtered row index,filtered column index,generated response
0,What product line is in the latest entry?,999,3,Fashion accessories,NaN,[3],NaN
1,On what date did the first transaction occur?,"17, 245, 450, 484, 496, 523, 567, 696, 829, 83...",7,1/1/2019,NaN,[7],NaN
2,What is the latest transaction date?,"158, 306, 473, 474, 643, 646, 671, 881, 883, 9...",7,3/30/2019,NaN,[7],NaN
3,what is the max rating given in home and lifes...,"2, 7, 19, 22, 25, 39, 40, 41, 54, 56, 58, 61, ...","3,10",9.9,NaN,"[10, 3]",NaN
4,How many transactions involved Male customers ...,"331, 464, 540, 708, 710","2,10",5,NaN,"[10, 2]",NaN


In [ ]:
def row_filtering():
    lst = updated_qa["filtered column index"]
    for clms in lst:
        
    

In [ ]:
column_data = read_excel_column(r"C:\Users\harme\Downloads\Hackathon - LPU_2025\TQ-29\updated_QA.xlsx", )

#### working on filtering for a single query

In [34]:
embedded_data = df["product line"].apply(lambda x: model.encode([x])[0])


In [35]:
embedded_data = np.array(embedded_data.tolist())

In [36]:
ratings = df["rating"].values.reshape(-1, 1)  # Reshape for concatenation


In [37]:
embedded_data = np.array(embedded_data.tolist())


In [38]:
ratings = df["rating"].values.reshape(-1, 1)  # Reshape for concatenation


In [39]:
vector_data = np.hstack((embedded_data, ratings))


In [40]:
# Determine the dimension for FAISS indexing
dimension = vector_data.shape[1]

In [41]:

# Create a FAISS index for L2-based similarity search
index = faiss.IndexFlatL2(dimension)

# Add vectors to the FAISS index
index.add(vector_data)

# Save the FAISS index for future use
faiss.write_index(index, "vector_store.index")

In [64]:
# List of query words
query_words = ["home", "lifestyle", "max", "given"]

# Initialize a set to store unique matching row indices
matching_row_indices = set()

for word in query_words:
    # Generate query embedding for the input word
    query_embedding = model.encode([word])[0]

    # Append a dummy rating value for comparison (matching dimensions)
    query_vector = np.append(query_embedding, [0.0]).reshape(1, -1)

    # Perform similarity search for top 200 matches
    D, I = index.search(query_vector, k=200)
    print(f"Word: {word}, Distances: {D}, Indexes: {I}")

    # Filter rows based on the threshold
    filtered_indices = [i for i, distance in enumerate(D[0]) if distance <= threshold]

    # Collect row indices for valid matches
    if filtered_indices:
        matching_row_indices.update(I[0][filtered_indices])

# Convert to a sorted list for better handling
matching_row_indices = sorted(list(matching_row_indices))
print("Matching row indices:", matching_row_indices)

# Retrieve the full matching rows from the DataFrame
matching_rows = df.iloc[matching_row_indices]
print("Matching rows:", matching_rows)




Word: home, Distances: [[17.406937 17.406937 17.406937 17.406937 17.522652 17.527887 17.527887
  17.527887 17.527887 17.556938 17.556938 17.556938 17.60545  17.60545
  17.674591 18.236935 18.236935 18.236935 18.236935 18.332651 18.332651
  18.332651 18.332651 18.337887 18.366938 18.415451 18.415451 18.415451
  18.415451 18.484589 18.484589 18.484589 19.086939 19.086939 19.086939
  19.086939 19.086939 19.16265  19.16265  19.16265  19.16265  19.16265
  19.167885 19.167885 19.167885 19.167885 19.196936 19.196936 19.196936
  19.196936 19.196936 19.24545  19.24545  19.314587 19.314587 19.956938
  19.956938 19.956938 19.956938 20.012653 20.012653 20.017889 20.017889
  20.04694  20.04694  20.04694  20.04694  20.095451 20.164593 20.164593
  20.164593 20.164593 20.846937 20.846937 20.846937 20.882652 20.887888
  20.887888 20.887888 20.887888 20.916939 20.916939 20.96545  20.96545
  20.96545  21.034592 21.034592 21.034592 21.756937 21.772652 21.772652
  21.772652 21.772652 21.772652 21.777887 21

For row filtering and retrieval we can create chunks based on rime casting 